In [6]:
import numpy as np
import os
from keras.layers import Conv3D, MaxPooling3D
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential
from keras.layers import Activation, ZeroPadding3D, TimeDistributed, LSTM, GRU, Reshape
#from keras.utils import plot_model
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import gc
from sklearn.metrics import classification_report, balanced_accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from itertools import cycle
from sklearn.preprocessing import LabelEncoder
import sklearn
import seaborn as sns
import tensorflow as tf
from keras import regularizers
from tensorflow.keras.utils import to_categorical
from prettytable import PrettyTable
from matplotlib import pyplot as plt

import time

# Noise Reduction 

In [2]:
import cv2
import os

def reduce_noise(input_folder, output_folder):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop through each file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".mp4"):  # Assuming all files are videos
            video_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            # Open the video file
            cap = cv2.VideoCapture(video_path)
            frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            fps = int(cap.get(cv2.CAP_PROP_FPS))

            # Create VideoWriter object
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

            # Process each frame
            for _ in range(frame_count):
                ret, frame = cap.read()
                if ret:
                    # Apply Gaussian blur for noise reduction
                    blurred_frame = cv2.GaussianBlur(frame, (5, 5), 0)
                    out.write(blurred_frame)

            # Release resources
            cap.release()
            out.release()

            print(f"Noise reduction completed for {filename}")

# Example usage:
input_folder = "D:\\New Dataset Lip Movement Projec\\FINAL YEAR PROJECT\\Noise reduction\\Dataset2.0"
output_folder = "D:\\New Dataset Lip Movement Projec\\FINAL YEAR PROJECT\\Noise reduction\\output"
reduce_noise(input_folder, output_folder)


Noise reduction completed for 0.mp4
Noise reduction completed for 1.mp4
Noise reduction completed for 10.mp4
Noise reduction completed for 2.mp4
Noise reduction completed for 3.mp4
Noise reduction completed for 4.mp4
Noise reduction completed for 5.mp4
Noise reduction completed for 6.mp4
Noise reduction completed for 7.mp4
Noise reduction completed for 8.mp4
Noise reduction completed for 9.mp4
Noise reduction completed for box.mp4
Noise reduction completed for fun.mp4
Noise reduction completed for good.mp4
Noise reduction completed for grow.mp4
Noise reduction completed for hero.mp4
Noise reduction completed for trust.mp4


# CNN


In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define constants
IMAGE_SIZE = (96, 72)
BATCH_SIZE = 16
EPOCHS = 100
NUM_CLASSES = 17
TRAIN_DATA_DIR = "D:\\New Dataset Lip Movement Projec\\FINAL YEAR PROJECT\\output_denoised_image_folder"

# Define data generator
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Load and preprocess the data
train_generator = datagen.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Define the CNN model
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Found 246 images belonging to 17 classes.


Found 52 images belonging to 17 classes.



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 94, 70, 64)        1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 47, 35, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 45, 33, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 22, 16, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 14, 128)       147584    
                                                                 
 max_pooling

In [70]:

start_time=time.time()
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE
)

# Stop the timer
end_time = time.time()

# Compute the computational time
comp_time = end_time - start_time

model.save('cnnown1.h5')
# Evaluate the model
train_loss, train_acc = model.evaluate(train_generator)
val_loss, val_acc = model.evaluate(validation_generator)

print(f'Training Accuracy: {train_acc * 100}')
print(f'Validation Accuracy: {val_acc * 100}')
print(f'Trainig time: {comp_time}')

Epoch 1/100
15/15 [==============================] - 3s 150ms/step - loss: 2.8505 - accuracy: 0.0478 - val_loss: 2.8309 - val_accuracy: 0.0625
Epoch 2/100
15/15 [==============================] - 2s 105ms/step - loss: 2.8375 - accuracy: 0.0652 - val_loss: 2.8264 - val_accuracy: 0.0833
Epoch 3/100
15/15 [==============================] - 2s 114ms/step - loss: 2.8305 - accuracy: 0.0652 - val_loss: 2.8177 - val_accuracy: 0.0625
Epoch 4/100
15/15 [==============================] - 2s 107ms/step - loss: 2.7735 - accuracy: 0.1087 - val_loss: 2.5745 - val_accuracy: 0.2083
Epoch 5/100
15/15 [==============================] - 2s 120ms/step - loss: 2.4676 - accuracy: 0.1792 - val_loss: 2.2378 - val_accuracy: 0.2292
Epoch 6/100
15/15 [==============================] - 2s 109ms/step - loss: 2.2445 - accuracy: 0.2130 - val_loss: 2.1215 - val_accuracy: 0.2292
Epoch 7/100
15/15 [==============================] - 2s 120ms/step - loss: 2.0027 - accuracy: 0.2783 - val_loss: 2.0716 - val_accuracy: 0.2917

c:\Users\ksaqu\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4/4 [==============================] - 0s 25ms/step - loss: 5.7941e-04 - accuracy: 1.0000
Training Accuracy: 100.0
Validation Accuracy: 100.0
Trainig time: 171.78111267089844


In [67]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# # Create an ImageDataGenerator object
# datagen = ImageDataGenerator(rescale=1./255)

# # Create a generator
# generator = datagen.flow_from_directory(
#     'D:\\New Dataset Lip Movement Projec\\FINAL YEAR PROJECT\\output_denoised_image_folder',
#     target_size=(150, 150),
#     batch_size=16,
#     class_mode='binary'
# )

# print(generator.batch_size)


Found 298 images belonging to 17 classes.
16


# Converting Normal Frames into GLCM

In [12]:
import os
import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops

def glcm_noise_reduction(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Compute the GLCM of the grayscale image
    glcm = graycomatrix(gray, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4])

    # Compute the mean of the GLCM
    mean_glcm = np.mean(glcm)

    # Subtract the mean from the original image to reduce noise
    reduced_noise_image = gray - mean_glcm

    return reduced_noise_image

# Specify the directory containing the folders with frames
root_dir = 'D:\\New Dataset Lip Movement Projec\\FINAL YEAR PROJECT\\Frames'

# Create a new folder to save all the noise-reduced images
new_root_dir = 'D:\\New Dataset Lip Movement Projec\\FINAL YEAR PROJECT\\output_denoised_image_folder'

# Loop over each folder (video)
for folder in os.listdir(root_dir):
    folder_path = os.path.join(root_dir, folder)
    new_folder_path = os.path.join(new_root_dir, folder)
    os.makedirs(new_folder_path, exist_ok=True)

    # Loop over each image (frame) in the folder
    for frame in os.listdir(folder_path):
        frame_path = os.path.join(folder_path, frame)

        # Read the image
        image = cv2.imread(frame_path)

        # Apply GLCM noise reduction
        reduced_noise_image = glcm_noise_reduction(image)

        # Save the noise-reduced image in the new folder
        cv2.imwrite(os.path.join(new_folder_path, frame), reduced_noise_image)

print('Noise reduction completed.')


KeyboardInterrupt: 